<a href="https://colab.research.google.com/github/Zernach/BetterReads-DS/blob/master/IP_Address_Test_GoogleBooks_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IP Address Proxy

In [0]:
!curl ipecho.net/plain

35.223.189.101

In [0]:
%%bash

chmod 777 /tmp
mkdir data
apt-get update --allow-unauthenticated 
apt-get update -y --fix-missing 
pip install selenium
apt-get install chromium-chromedriver -y --fix-missing
pip install joblib
apt-get update --fix-missing

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [91.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [8,505 B]
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubunt

In [0]:
import requests
from bs4 import BeautifulSoup

URL = 'https://api.ipify.org?format=json'
req = requests.get(URL) # .json()
soup = BeautifulSoup(req.text, "lxml")
ip = soup.find('p').text
print('Your IP ', ip)

# Your IP  {"ip":"35.197.23.94"} # Note: This will be different on your VM

Your IP  {"ip":"35.223.189.101"}


In [0]:
import requests
from bs4 import BeautifulSoup

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
URL = 'https://api.ipify.org?format=json'
proxy = {'https': '200.255.122.174:8080'}
req = requests.get(URL, proxies = proxy, headers = headers) # .json()
soup = BeautifulSoup(req.text, "lxml")
ip = soup.find('p').text
print('IP from scrape w/proxy ', ip)

# Output
# IP from scrape w/proxy  {"ip":"200.255.122.174"} # Tests proxy

ProxyError: ignored

In [0]:
from lxml.html import fromstring
import requests
from itertools import cycle
import traceback

def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies


#If you are copy pasting proxy ips, put in the list below
#proxies = ['121.129.127.209:80', '124.41.215.238:45169', '185.93.3.123:8080', '194.182.64.67:3128', '106.0.38.174:8080', '163.172.175.210:3128', '13.92.196.150:8080']
proxies = get_proxies()
proxy_pool = cycle(proxies)

url = 'https://httpbin.org/ip'
for i in range(1,11):
    #Get a proxy from the pool
    proxy = next(proxy_pool)
    print("Request #%d"%i)
    try:
        response = requests.get(url,proxies={"http": proxy, "https": proxy})
        print(response.json())
    except:
        #Most free proxies will often get connection errors. You will have retry the entire request using another proxy to work. 
        #We will just skip retries as its beyond the scope of this tutorial and we are only downloading a single url 
        print("Skipping. Connnection error")

Request #1
Skipping. Connnection error
Request #2
Skipping. Connnection error
Request #3
Skipping. Connnection error
Request #4
Skipping. Connnection error
Request #5
Skipping. Connnection error
Request #6
Skipping. Connnection error
Request #7
{'origin': '81.217.151.218'}
Request #8
{'origin': '182.23.81.82'}
Request #9
Skipping. Connnection error
Request #10
{'origin': '182.23.81.82'}


In [0]:
pip install scrapy-rotating-proxies

# Code for GoogleBooks API

In [0]:
# Import Statements
import requests
import json
import pickle
import pandas as pd
import numpy as np
import time
import math
import urllib
#from secrets import GOOGLE_KEY

#publishers = pd.read_csv("https://github.com/Lambda-School-Labs/betterreads-ds/blob/New_Data/notebooks/publishers.csv")
publishers = pd.read_csv("https://ryan.zernach.com/wp-content/uploads/2020/04/publishers.csv")
publishers['publisher'] = publishers['publisher'].apply(str)
publishers['publishers'] = publishers['publisher'].apply(urllib.parse.quote)
publishers_list = list(publishers['publishers'])

publishers_list_test = publishers_list[1100:1200]

In [0]:
%%time

cols = ['id', 'title', 'authors', 'publisher', 'publishedDate', 'description',
        'isbn', 'pageCount', 'categories', 'thumbnail', 'smallThumbnail',
        'language', 'webReaderLink', 'textSnippet', 'isEbook', 'averageRating',
        'maturityRating', 'ratingsCount', 'subtitle']

df = pd.DataFrame()

for publisher in publishers_list_test:

        response = requests.get('https://www.googleapis.com/books/v1/volumes?q=inpublisher' + publisher + '&maxResults=' + '40')
        data = json.loads(response.text)

        try:

          if data['totalItems'] == 0:
            pass
          else:

            # Pagination Loop
            # Insert code for loop

            if data['totalItems'] >= 40:
              end_of_loop = 40
            else:
              end_of_loop = data['totalItems']

              for item in range(0, end_of_loop):

                try:
                  book_id = data['items'][item][cols[0]]
                except Exception:
                  book_id = math.nan

                try:
                  title = data['items'][item]['volumeInfo'][cols[1]]
                except Exception:
                  title = math.nan

                try:
                  authors = data['items'][item]['volumeInfo'][cols[2]][0]
                except Exception:
                  authors = math.nan

                try:
                  publisher = data['items'][item]['volumeInfo'][cols[3]]
                except Exception:
                  publisher = math.nan

                try:
                  publishedDate = data['items'][item]['volumeInfo'][cols[4]]
                except Exception:
                  publishedDate = math.nan
              
                try:
                  description = data['items'][item]['volumeInfo'][cols[5]]
                except Exception:
                  description = math.nan
              
                try:
                  isbn = data['items'][item]['volumeInfo']['industryIdentifiers'][1]['identifier']
                except Exception:
                  isbn = math.nan

                try:
                  pageCount = data['items'][item]['volumeInfo'][cols[7]]
                except Exception:
                  pageCount = math.nan

                try:
                  categories = data['items'][item]['volumeInfo'][cols[8]][0]
                except Exception:
                  categories = math.nan

                try:
                  thumbnail = data['items'][item]['volumeInfo']['imageLinks'][cols[9]]
                except Exception:
                  thumbnail = math.nan

                try:
                  smallThumbnail = data['items'][item]['volumeInfo']['imageLinks'][cols[10]]
                except Exception:
                  smallThumbnail = math.nan

                try:
                  language = data['items'][item]['volumeInfo'][cols[11]]
                except Exception:
                  language = math.nan

                try:
                  webReaderLink = data['items'][item]['accessInfo'][cols[12]]
                except Exception:
                  webReaderLink = math.nan

                try:
                  textSnippet = data['items'][item][cols[13]]
                except Exception:
                  textSnippet = math.nan

                try:
                  isEbook = data['items'][item]['saleInfo'][cols[14]]
                except Exception:
                  isEbook = math.nan

                try:
                  averageRating = data['items'][item]['volumeInfo'][cols[15]]
                except Exception:
                  averageRating = math.nan

                try:
                  maturityRating = data['items'][item]['volumeInfo'][cols[16]]
                except Exception:
                  maturityRating = math.nan

                try:
                  ratingsCount = data['items'][item]['volumeInfo'][cols[17]]
                except Exception:
                  ratingsCount = math.nan

                try:
                  subtitle = data['items'][item]['volumeInfo'][cols[18]]
                except Exception:
                  subtitle = math.nan

                book_data = pd.DataFrame({cols[0]: book_id,
                                        cols[1]: title,
                                        cols[2]: authors,
                                        cols[3]: publisher,
                                        cols[4]: publishedDate,
                                        cols[5]: description,
                                        cols[6]: isbn,
                                        cols[7]: pageCount,
                                        cols[8]: categories,
                                        cols[9]: thumbnail,
                                        cols[10]: smallThumbnail,
                                        cols[11]: language,
                                        cols[12]: webReaderLink,
                                        cols[13]: textSnippet,
                                        cols[14]: isEbook,
                                        cols[15]: averageRating,
                                        cols[16]: maturityRating,
                                        cols[17]: ratingsCount,
                                        cols[18]: subtitle,
                                        }, index=[0])
                
                frames = [df, book_data]
                df = pd.concat(frames)
                checkpoint = publisher

        except Exception:
          pass

df = df.reset_index()
df.head(100)

CPU times: user 2.56 s, sys: 72.7 ms, total: 2.63 s
Wall time: 42.2 s
